In [ ]:
!pip install qiskit qiskit-machine-learning scikit-learn pandas numpy matplotlib seaborn
!pip install --upgrade qiskit-aer

# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

from qiskit.circuit.library import PauliFeatureMap, ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from qiskit_aer import AerSimulator

algorithm_globals.random_seed = 42
np.random.seed(42)

print("QSVM vs Classical SVM for ECG Classification")

class ECGDataGenerator:
    def __init__(self, random_state=42):
        np.random.seed(random_state)
        self.feature_names = ['Ventricular_Rate','Atrial_Rate','QRS_Duration','Q_Interval',
                              'QT_Corrected','R_Axis','T_Axis','QRS_Count',
                              'Q_Onset','Q_Offset','T_Offset']
        self.classes = ['AFIB','GSVT','SB','SR']
        self.class_desc = {'AFIB':'Atrial Fibrillation','GSVT':'Supraventricular Tachycardia',
                           'SB':'Sinus Bradycardia','SR':'Sinus Rhythm (Normal)'}
    
    def generate_class_features(self, class_name, n_samples):
        if class_name=="AFIB":
            params = [(110,20),(150,30),(100,15),(120,20),(420,40),(60,30),(40,25),(90,10),(50,10),(350,30),(400,40)]
        elif class_name=="GSVT":
            params = [(160,25),(160,25),(90,12),(100,15),(380,35),(75,25),(50,20),(110,15),(45,8),(320,25),(370,30)]
        elif class_name=="SB":
            params = [(45,8),(45,8),(95,10),(130,18),(440,35),(65,20),(35,15),(45,5),(55,12),(380,35),(430,40)]
        else:
            params = [(75,12),(75,12),(95,10),(120,15),(410,30),(70,20),(45,18),(75,8),(50,10),(350,25),(400,35)]
        features = [[np.random.normal(m,s) for m,s in params] for _ in range(n_samples)]
        return np.array(features)
    
    def generate_dataset(self, total_samples=2000):
        class_props = {'AFIB':0.15,'GSVT':0.25,'SB':0.20,'SR':0.40}
        features, labels = [], []
        print("Generating ECG dataset...")
        for cls, prop in class_props.items():
            n_samp = int(total_samples*prop)
            fts = self.generate_class_features(cls,n_samp)
            features.extend(fts)
            labels.extend([cls]*n_samp)
            print(f"{cls} ({self.class_desc[cls]}): {n_samp} samples")
        return np.array(features), np.array(labels)

class QuantumClassicalComparator:
    def __init__(self):
        self.label_encoder=LabelEncoder()
        self.scaler=StandardScaler()
        self.backend=AerSimulator()
        self.results = {}
        
    def preprocess_data(self,X,y):
        y_enc=self.label_encoder.fit_transform(y)
        X_train,X_test,y_train,y_test=train_test_split(X,y_enc,test_size=0.2,random_state=42,stratify=y_enc)
        X_train_scale=self.scaler.fit_transform(X_train)
        X_test_scale=self.scaler.transform(X_test)
        return X_train_scale,X_test_scale,y_train,y_test
    
    def train_classical_svm(self,X_train,X_test,y_train,y_test):
        clf=SVC(kernel='rbf',random_state=42)
        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)
        acc=accuracy_score(y_test,y_pred)
        return acc,y_pred
    
    def train_quantum_svm(self,X_train,X_test,y_train,y_test,feature_map_name):
        n_feat=X_train.shape[1]
        if feature_map_name=='PauliX':
            fmap=PauliFeatureMap(feature_dimension=n_feat,reps=1,paulis=['X'])
        elif feature_map_name=='PauliZ':
            fmap=PauliFeatureMap(feature_dimension=n_feat,reps=1,paulis=['Z'])
        else:
            raise ValueError('Unsupported feature map.')
        qkernel=QuantumKernel(feature_map=fmap,quantum_instance=self.backend)
        qsvm=QSVC(quantum_kernel=qkernel)
        subset_size = min(150, len(X_train))
        idxs = np.random.choice(len(X_train),subset_size,replace=False)
        X_train_sub, y_train_sub = X_train[idxs], y_train[idxs]
        qsvm.fit(X_train_sub,y_train_sub)
        y_pred=qsvm.predict(X_test)
        acc=accuracy_score(y_test,y_pred)
        return acc,y_pred
    
    def run_experiment(self,X,y):
        X_train,X_test,y_train,y_test = self.preprocess_data(X,y)
        qubits=[3,5]
        feature_maps=['PauliX','PauliZ']

        for n_qubits in qubits:
            pca=PCA(n_components=n_qubits,random_state=42)
            X_train_pca=pca.fit_transform(X_train)
            X_test_pca=pca.transform(X_test)
            classical_acc,classical_pred=self.train_classical_svm(X_train_pca,X_test_pca,y_train,y_test)
            best_quantum_acc=0;best_map=None;best_quantum_pred=None
            for fmap in feature_maps:
                quantum_acc, quantum_pred = self.train_quantum_svm(X_train_pca, X_test_pca, y_train, y_test, fmap)
                if quantum_acc>best_quantum_acc:
                    best_quantum_acc=quantum_acc
                    best_map=fmap
                    best_quantum_pred=quantum_pred
            print(f"Qubits: {n_qubits} | Classical: {classical_acc:.4f} | Quantum: {best_quantum_acc:.4f} | Map: {best_map}")
            self.results[n_qubits]={'classical_accuracy': classical_acc,'quantum_accuracy': best_quantum_acc,
                                   'best_feature_map': best_map,'classical_pred': classical_pred,'quantum_pred': best_quantum_pred,
                                   'pca_variance': pca.explained_variance_ratio_.sum()}
        self.y_test = y_test
        return self.results

def plot_results(results, y_test, classes):
    sns.set(style='whitegrid')
    qubits = list(results.keys())
    classical_acc = [results[q]['classical_accuracy'] for q in qubits]
    quantum_acc = [results[q]['quantum_accuracy'] for q in qubits]
    gaps = [c - q for c, q in zip(classical_acc, quantum_acc)]
    
    plt.figure(figsize=(10,6))
    width=0.35
    x=np.arange(len(qubits))
    plt.bar(x - width/2, classical_acc, width, label='Classical SVM', color='blue')
    plt.bar(x + width/2, quantum_acc, width, label='Quantum SVM', color='red')
    plt.xticks(x, [str(q) for q in qubits])
    plt.xlabel('Number of Qubits')
    plt.ylabel('Accuracy')
    plt.title('QSVM vs Classical SVM')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(6,4))
    plt.bar(qubits,gaps,color='orange')
    plt.xlabel('Number of Qubits')
    plt.ylabel('Performance Gap (Classical - Quantum)')
    plt.title('Performance Gap')
    plt.show()
    
    best_qubits = qubits[np.argmax(classical_acc)]
    best_result = results[best_qubits]
    cm_classical = confusion_matrix(y_test, best_result['classical_pred'])
    cm_quantum = confusion_matrix(y_test, best_result['quantum_pred'])
    
    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    sns.heatmap(cm_classical, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'Classical SVM Confusion Matrix ({best_qubits} Qubits)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    plt.subplot(1,2,2)
    sns.heatmap(cm_quantum, annot=True, fmt='d', cmap='Reds', xticklabels=classes, yticklabels=classes)
    plt.title(f'Quantum SVM Confusion Matrix ({best_qubits} Qubits)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    plt.tight_layout()
    plt.show()

if __name__=="__main__":
    ecg_gen = ECGDataGenerator()
    X,y = ecg_gen.generate_dataset(2000)
    comparator = QuantumClassicalComparator()
    results = comparator.run_experiment(X,y)
    plot_results(results, comparator.y_test, ecg_gen.classes)
